### Extract Rooms from IFC and plot them

In [1]:
import ifcopenshell
import re
import networkx as nx
import pandas as pd
import plotly.graph_objs as go
import csv
from plotly.subplots import make_subplots

def load_ifc_file(file_path):
    """Load the IFC file."""
    return ifcopenshell.open(file_path)

def filter_ifcspaces_by_storey(spaces, storey_name):
    """Filter IfcSpaces by the given storey name."""
    filtered_spaces = []
    for space in spaces:
        for rel in space.Decomposes:
            if rel.is_a("IfcRelAggregates") and rel.RelatingObject.is_a("IfcBuildingStorey"):
                if rel.RelatingObject.Name == storey_name:
                    filtered_spaces.append(space)
                    break
    return filtered_spaces

def filter_spaces_by_name(spaces):
    """Filter IfcSpaces by name containing only digits (1-5 digits)."""
    digit_pattern = re.compile(r'^\d{1,5}$')
    filtered_spaces = []
    for space in spaces:
        if digit_pattern.match(space.Name):
            filtered_spaces.append(space)
    return filtered_spaces

# -- MAIN --

# Load the IFC file
ifc_file = load_ifc_file("Hus28_test.ifc")
print("IFC file opened successfully.")

# Define the storey name to filter
storey_name = "Plan 10"

# Find all IfcSpaces
all_spaces = ifc_file.by_type("IfcSpace")
num_spaces = len(all_spaces)
print(f"Number of IfcSpaces found: {num_spaces}")

# Filter spaces by the specified storey name
spaces = filter_ifcspaces_by_storey(all_spaces, storey_name)
num_filtered_spaces = len(spaces)
print(f"Number of IfcSpaces matching storey '{storey_name}': {num_filtered_spaces}")

# Further filter spaces by names containing only digits (1-5 digits)
spaces = filter_spaces_by_name(spaces)
num_filtered_by_name = len(spaces)
print(f"Number of IfcSpaces with valid names: {num_filtered_by_name}")

# Create a NetworkX graph
G = nx.Graph()

# Add nodes to the graph
for space in spaces:
    node_label = f"Room_{space.Name}"
    global_id = space.GlobalId  # Extract the GlobalId from the IfcSpace object
    G.add_node(node_label, color='yellow', GlobalId=global_id)

# Create the Plotly graph
pos = nx.spring_layout(G)  # Positions for all nodes

# Extract node positions
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]
node_text = list(G.nodes())  # Only the node label (Room_XXX) will be displayed

# Create node trace
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=node_text,
    textposition='top center',
    hoverinfo='text',
    marker=dict(
        color='yellow',
        size=20,
        line=dict(width=2)
    )
)


# Create edge trace
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=1, color='#888'),
    hoverinfo='none',
    mode='lines'
)

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Network graph of IFC Spaces',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

# Show the figure
fig.show()


IFC file opened successfully.
Number of IfcSpaces found: 193
Number of IfcSpaces matching storey 'Plan 10': 37
Number of IfcSpaces with valid names: 30


### Add direct Contact between Rooms

In [2]:
# Load the CSV file
csv_file = "Output01_RoomToRoom_BySeparationLine.csv"

# Manually read and parse the CSV file
edges = []
with open(csv_file, 'r') as file:
    for line in file:
        parts = line.strip().split(';')
        main_room_global_id = parts[0]
        if len(parts) > 1:
            connected_rooms_global_ids = parts[1].split(',')
            for neighbor_global_id in connected_rooms_global_ids:
                edges.append((main_room_global_id, neighbor_global_id))

# Add edges to the graph
for edge in edges:
    # Find the corresponding node names based on GlobalId
    room1 = None
    room2 = None

    for node in G.nodes:
        if G.nodes[node]["GlobalId"] == edge[0]:
            room1 = node
        if G.nodes[node]["GlobalId"] == edge[1]:
            room2 = node

    if room1 and room2:
        G.add_edge(room1, room2, Access="Direct")

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color='blue' if edge[2]['Access'] == 'Direct' else 'black'),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color='yellow', size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


### Add Contact via Doors

In [3]:
# Load the second CSV file
csv_file_doors = "Output02_RoomToRoom_ByDoors.csv"

# Manually read and parse the second CSV file
door_edges = []
with open(csv_file_doors, 'r') as file:
    for line in file:
        parts = line.strip().split(';')
        door_global_id = parts[0]
        if len(parts) > 1:
            connected_rooms_global_ids = parts[1].split(',')
            if len(connected_rooms_global_ids) > 2:
                print(f"Door {door_global_id} connects more than two rooms: {connected_rooms_global_ids}")
            for i in range(len(connected_rooms_global_ids) - 1):
                for j in range(i + 1, len(connected_rooms_global_ids)):
                    door_edges.append((connected_rooms_global_ids[i], connected_rooms_global_ids[j]))

# Add door edges to the graph
for edge in door_edges:
    # Find the corresponding node names based on GlobalId
    room1 = None
    room2 = None

    for node in G.nodes:
        if G.nodes[node]["GlobalId"] == edge[0]:
            room1 = node
        if G.nodes[node]["GlobalId"] == edge[1]:
            room2 = node

    if room1 and room2:
        G.add_edge(room1, room2, Access="Door")

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = 'blue' if edge[2]['Access'] == 'Direct' else 'green' if edge[2]['Access'] == 'Door' else 'black'
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color='yellow', size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


Door 0aNC3YsFzEZf8uB9g1nSdw connects more than two rooms: ['0aNC3YsFzEZf8uB9g1nSfO', '0aNC3YsFzEZf8uB9g1nSfO', '3V7W2RscLEExyQLuCnt1om']


### Add Contact via Windows

In [4]:
# Load the third CSV file
csv_file_windows = "Output03_RoomToRoom_ByWindows.csv"

# Manually read and parse the third CSV file
window_edges = set()
with open(csv_file_windows, 'r') as file:
    for line in file:
        parts = line.strip().split(';')
        window_global_id = parts[0]
        if len(parts) > 1:
            connected_rooms_global_ids = parts[1].split(',')
            if len(connected_rooms_global_ids) > 1:
                for i in range(len(connected_rooms_global_ids)):
                    for j in range(i + 1, len(connected_rooms_global_ids)):
                        edge = (connected_rooms_global_ids[i], connected_rooms_global_ids[j])
                        # Ensure bidirectional uniqueness
                        if edge[0] != edge[1] and (edge not in window_edges and (edge[1], edge[0]) not in window_edges):
                            window_edges.add(edge)

# Add window edges to the graph
for edge in window_edges:
    room1 = None
    room2 = None

    for node in G.nodes:
        if G.nodes[node]["GlobalId"] == edge[0]:
            room1 = node
        if G.nodes[node]["GlobalId"] == edge[1]:
            room2 = node

    if room1 and room2:
        G.add_edge(room1, room2, Access="Window")

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = 'blue' if edge[2]['Access'] == 'Direct' else 'green' if edge[2]['Access'] == 'Door' else 'red' if edge[2]['Access'] == 'Window' else 'black'
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color='yellow', size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


### Add Doors

In [5]:
# Load the second CSV file
csv_file_doors = "Output02_RoomToRoom_ByDoors.csv"

# Manually read and parse the second CSV file
door_edges = []
doors = set()
with open(csv_file_doors, 'r') as file:
    for line in file:
        parts = line.strip().split(';')
        door_global_id = parts[0]
        connected_rooms_global_ids = parts[1].split(',') if len(parts) > 1 else []
        
        # Assume that we can retrieve the door object from the GlobalId
        door = next((d for d in ifc_file.by_type("IfcDoor") if d.GlobalId == door_global_id), None)
        
        if door:
            door_oid = door.id()
            doors.add((f"Door_{door_oid}", door_global_id, door_oid))
        
            for room_global_id in connected_rooms_global_ids:
                door_edges.append((door_oid, room_global_id))

# Add door nodes to the graph
for door_name, door_global_id, door_oid in doors:
    G.add_node(door_name, color='blue', GlobalId=door_global_id, OID=door_oid)

# Add door edges to the graph
for edge in door_edges:
    door = f"Door_{edge[0]}"
    room = None

    # Find the corresponding room node by GlobalId
    for node in G.nodes:
        if G.nodes[node]["GlobalId"] == edge[1]:
            room = node
            break

    if room:
        G.add_edge(door, room, Category="ContainedIn")

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = 'blue' if edge[2].get('Access') == 'Direct' else 'orange' if edge[2].get('Access') == 'Door' else 'red' if edge[2].get('IsConnected') else 'green' if edge[2].get('Category') == 'ContainedIn' else 'black'
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

# Adjust the node text to display only the node name without the OID in brackets
node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes],  # Display only the node name
                        mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color=[G.nodes[node]['color'] for node in G.nodes], size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()

### Add Windows

In [6]:
import ifcopenshell

# Load the IFC file
ifc_file = ifcopenshell.open("Hus28_test.ifc")  # Ersetze durch den tatsächlichen Pfad zur IFC-Datei

# Load the third CSV file
csv_file_windows = "Output03_RoomToRoom_ByWindows.csv"

# Manually read and parse the third CSV file
window_edges = []
windows = set()
with open(csv_file_windows, 'r') as file:
    for line in file:
        parts = line.strip().split(';')
        window_global_id = parts[0]
        connected_rooms_global_ids = parts[1].split(',') if len(parts) > 1 else []
        
        # Retrieve the OID using the window's id method
        window = next((w for w in ifc_file.by_type("IfcWindow") if w.GlobalId == window_global_id), None)
        
        if window:
            window_oid = window.id()
            windows.add((f"Window_{window_oid}", window_global_id, window_oid))
        
            for room_global_id in connected_rooms_global_ids:
                window_edges.append((window_oid, room_global_id))

# Add window nodes to the graph
for window_name, window_global_id, window_oid in windows:
    G.add_node(window_name, color='purple', GlobalId=window_global_id, OID=window_oid)

# Add window edges to the graph
for edge in window_edges:
    window = f"Window_{edge[0]}"
    room = None

    # Find the corresponding room node by GlobalId
    for node in G.nodes:
        if G.nodes[node]["GlobalId"] == edge[1]:
            room = node
            break

    if room:
        G.add_edge(window, room, Category="ContainedIn")

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = 'blue' if edge[2].get('Access') == 'Direct' else 'orange' if edge[2].get('Access') == 'Door' else 'purple' if edge[2].get('Category') == 'ContainedIn' else 'black'
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

# Adjust the node text to display only the node name without the OID in brackets
node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes],  # Display only the node name
                        mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color=[G.nodes[node]['color'] for node in G.nodes], size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


## Add Walls

In [7]:
def get_storey(element):
    """Get the storey of the given element."""
    for rel in element.ContainedInStructure:
        if rel.is_a("IfcRelContainedInSpatialStructure") and rel.RelatingStructure.is_a("IfcBuildingStorey"):
            return rel.RelatingStructure
    return None

# Find all IfcWalls
all_walls = ifc_file.by_type("IfcWall")
num_walls = len(all_walls)
print(f"Number of IfcWalls found: {num_walls}")

# Filter walls by the specified storey name
walls_in_storey = []
for wall in all_walls:
    storey = get_storey(wall)
    if storey and storey.Name == storey_name:
        walls_in_storey.append((wall.id(), wall.GlobalId))

num_filtered_walls = len(walls_in_storey)
print(f"Number of IfcWalls matching storey '{storey_name}': {num_filtered_walls}")

# Add wall nodes to the graph
for wall_oid, wall_global_id in walls_in_storey:
    node_label = f"Wall_{wall_oid}"
    G.add_node(node_label, color='grey', GlobalId=wall_global_id, OID=wall_oid)

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = 'blue' if edge[2].get('Access') == 'Direct' else 'orange' if edge[2].get('Access') == 'Door' else 'purple' if edge[2].get('IsConnected') else 'black'
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color=[G.nodes[node]['color'] for node in G.nodes], size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


Number of IfcWalls found: 638
Number of IfcWalls matching storey 'Plan 10': 138


### Connection Room-Walls

In [8]:
import csv

# Helper function to get wall OID from GlobalId
def get_wall_oid_by_guid(walls, guid):
    for wall in walls:
        if wall.GlobalId == guid:
            return wall.id()
    return None

# Load the CSV file
csv_file_spaces_walls = "Output04_RoomBoundingWalls.csv"

# Manually read and parse the CSV file
with open(csv_file_spaces_walls, 'r') as file:
    reader = csv.reader(file, delimiter=';')
    for row in reader:
        space_global_id = row[0]  # The GlobalId of the room
        wall_guids = row[1].split(',')  # The GlobalIds of the walls

        # Find the room node by GlobalId
        space_node = None
        for node in G.nodes:
            if G.nodes[node].get("GlobalId") == space_global_id:
                space_node = node
                break

        if space_node:
            for wall_guid in wall_guids:
                wall_oid = get_wall_oid_by_guid(all_walls, wall_guid)
                if wall_oid is not None:
                    wall_node = f"Wall_{wall_oid}"
                    if wall_node in G:
                        G.add_edge(space_node, wall_node, Category="ContainedIn")


# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = 'blue' if edge[2].get('Access') == 'Direct' else 'orange' if edge[2].get('Access') == 'Door' else 'purple' if edge[2].get('Access') == 'Window' else 'grey' if edge[2].get('Category') == 'ContainedIn' else 'black'
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color=[G.nodes[node]['color'] for node in G.nodes], size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


### Connections Walls - Walls

In [10]:
import csv

# Helper function to get wall OID from GlobalId
def get_wall_oid_by_guid(walls, guid):
    for wall in walls:
        if wall.GlobalId == guid:
            return wall.id()
    return None

# Load the CSV file
csv_file_walls_to_walls = "Output06_Wall_Adjacancy.csv"

# Manually read and parse the CSV file
with open(csv_file_walls_to_walls, 'r') as file:
    reader = csv.reader(file, delimiter=';')
    for row in reader:
        primary_wall_guid = row[0]  # The GlobalId of the primary wall
        connected_wall_guids = row[1].split(',')  # The GlobalIds of connected walls

        # Get OID for the primary wall
        primary_wall_oid = get_wall_oid_by_guid(all_walls, primary_wall_guid)
        if primary_wall_oid is not None:
            primary_wall_node = f"Wall_{primary_wall_oid}"
            
            for connected_wall_guid in connected_wall_guids:
                # Get OID for each connected wall
                connected_wall_oid = get_wall_oid_by_guid(all_walls, connected_wall_guid)
                if connected_wall_oid is not None:
                    connected_wall_node = f"Wall_{connected_wall_oid}"
                    if primary_wall_node in G and connected_wall_node in G:
                        G.add_edge(primary_wall_node, connected_wall_node, Category="IsConnected")


# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = ('blue' if edge[2].get('Access') == 'Direct' else
             'orange' if edge[2].get('Access') == 'Door' else
             'purple' if edge[2].get('Access') == 'Window' else
             'grey' if edge[2].get('ContainedIn') else
             'red' if edge[2].get('Category') == 'IsConnected' else
             'black')
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color=[G.nodes[node]['color'] for node in G.nodes], size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


### Host of Windows and Doors

In [12]:
import csv

# Helper function to get element OID from GUID
def get_element_oid_by_guid(elements, guid):
    for element in elements:
        if element.GlobalId == guid:
            return element.id()
    return None

# Load the CSV file
csv_file_host_elements = "Output05_Hosts_of_WindowsAndDoors.csv"

# Manually read and parse the CSV file
with open(csv_file_host_elements, 'r') as file:
    reader = csv.reader(file, delimiter=';')
    for row in reader:
        element_type = row[0]  # The category of the element (e.g., "IfcDoor" or "IfcWindow")
        element_guid = row[1]  # The GlobalId of the element
        wall_guid = row[2]     # The GlobalId of the wall in which the element is hosted

        # Get OID for the element
        element_oid = get_element_oid_by_guid(ifc_file.by_type(element_type), element_guid)
        if element_oid is not None:
            element_node = f"{element_type.split('Ifc')[-1]}_{element_oid}"
            
            # Get OID for the wall
            wall_oid = get_element_oid_by_guid(all_walls, wall_guid)
            if wall_oid is not None:
                wall_node = f"Wall_{wall_oid}"
                
                if element_node in G and wall_node in G:
                    G.add_edge(element_node, wall_node, Category="HostedBy")


# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = ('blue' if edge[2].get('Access') == 'Direct' else
             'orange' if edge[2].get('Access') == 'Door' else
             'purple' if edge[2].get('Access') == 'Window' else
             'grey' if edge[2].get('ContainedIn') else
             'red' if edge[2].get('ConnectedWith') else
             'green' if edge[2].get('Category') == 'HostedBy' else
             'black')
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color=[G.nodes[node]['color'] for node in G.nodes], size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


In [11]:
import plotly.io as pio

# Save Plotly to HTML
pio.write_html(fig, 'graph.html', auto_open=True)

In [12]:
import pickle

# Save NetworkX to pickle
with open('network_graph.pkl', 'wb') as f:
    pickle.dump(G, f)